In [ ]:
import pandas as pd
import datetime
import random
import math
import collections

period_in_seconds = 5*60  # 5-min period
devices = 10

# start = datetime.datetime.now()
start = datetime.datetime(2018, 1, 1)

# Generate hits values for n devices in a time period
gen_data = (start + datetime.timedelta(seconds=random.randrange(period_in_seconds)) for i in range (0, devices))
hits = collections.Counter(gen_data)

# Generate 1 period TimeSeries of zero values, and add our hits to each time bucket
ts = pd.Series(0, index=pd.date_range(start, start+datetime.timedelta(seconds=period_in_seconds-1), freq='1s'))
ts = ts.add(pd.Series((i for i in hits.values()), index=hits.keys()), fill_value=0)




In [ ]:
# plotting-related imports
import matplotlib.pylab as plt
%matplotlib inline

plt.plot(ts, color='blue', label='hits')